# 10 Keep latest VRS trip data

The vrs data contain different status updates, when the PT vehicle will arrive. Because of that, there are duplicates of information, which need to be removed. 

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib

In [2]:
data_month_double = '07'

In [3]:
#data = pd.read_parquet('/Volumes/T7/Master/Processed Data/'+data_month_double+'/10_second_merge.parquet', engine='pyarrow')
#data = pd.read_csv('../Data/10_second_merge.csv')
data = pd.read_csv('../Data/09_second_merge.csv')

/tmp/ipykernel_13640/3478774994.py:3: DtypeWarning: Columns (4,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../../../../data/active/research-data/students/akotsovolos/Data/10_second_merge_NEU_NEU.csv')


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71174472 entries, 0 to 71174471
Data columns (total 30 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   Unnamed: 0.1              int64 
 1   Unnamed: 0                int64 
 2   route_id                  int64 
 3   agency_id                 int64 
 4   route_short_name          object
 5   route_type                int64 
 6   route_type_name           object
 7   agency_name               object
 8   service_id                int64 
 9   trip_id                   object
 10  trip_headsign             object
 11  direction_id              int64 
 12  shape_id                  int64 
 13  stop_id                   int64 
 14  actual_arrival_time       object
 15  actual_departure_time     object
 16  vrs_timestamp             int64 
 17  stop_sequence             int64 
 18  stop_headsign             object
 19  stop_name                 object
 20  station_point             object
 21  buffer

## Remove

In [5]:
# sort data
sorted_data = data.sort_values(by=['trip_id', 'scheduled_arrival_time'])

In [6]:
sorted_data

,Unnamed: 0.1,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,...,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday,transfer_stop
93788,93788,93788,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
93789,93789,93789,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
93790,93790,93790,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
93791,93791,93791,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
93792,93792,93792,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116078,116078,116078,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.119586000000005 50.751904),"POLYGON ((7.122338292584601 50.75223811660463,...",2023-07-02 19:48:00,2023-07-02 19:48:00,0 days,0 days,1,2023-07-02,Sunday,False
116084,116084,116084,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.119586000000005 50.751904),"POLYGON ((7.122338292584601 50.75223811660463,...",2023-07-02 19:48:00,2023-07-02 19:48:00,0 days,0 days,1,2023-07-02,Sunday,False
116090,116090,116090,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.119586000000005 50.751904),"POLYGON ((7.122338292584601 50.75223811660463,...",2023-07-02 19:48:00,2023-07-02 19:48:00,0 days,0 days,1,2023-07-02,Sunday,False
116096,116096,116096,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.119586000000005 50.751904),"POLYGON ((7.122338292584601 50.75223811660463,...",2023-07-02 19:48:00,2023-07-02 19:48:00,0 days,0 days,1,2023-07-02,Sunday,False


In [7]:
# get first trip id
sorted_data.iloc[0]['trip_id']

'1-18-001-697.1.13:034800-39-256_56C5151C-E402-4E42-BED8-AFF1013D62C7'

In [8]:
# get data with first trip id
first_id = data[data['trip_id'] == sorted_data.iloc[0]['trip_id']]

In [9]:
# drop unneccessary columns for better readability
first_id.drop(columns=[ 'route_id', 'route_short_name', 'route_type_name', 'agency_name', 'transfer_stop'], inplace=True)

/tmp/ipykernel_13640/4044374218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_id.drop(columns=[ 'route_id', 'route_short_name', 'route_type_name', 'agency_name', 'transfer_stop'], inplace=True)


In [10]:
first_id

,Unnamed: 0.1,Unnamed: 0,agency_id,route_type,service_id,trip_id,trip_headsign,direction_id,shape_id,stop_id,...,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday
93788,93788,93788,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93789,93789,93789,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93790,93790,93790,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93791,93791,93791,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93792,93792,93792,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93793,93793,93793,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93794,93794,93794,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93795,93795,93795,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93796,93796,93796,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday
93797,93797,93797,1,0,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,8985,697,...,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday


In [11]:
# look for duplicates in scheduled arrival time
#duplicates = df[df['scheduled_arrival_time'].duplicated()]

In [12]:
# drop columns for better readability
#duplicates.drop(columns=['agency_id', 'route_type', 'shape_id', 'date'], inplace=True)
first_id.drop(columns=['agency_id', 'route_type', 'shape_id', 'date'], inplace=True)

/tmp/ipykernel_13640/3560580587.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_id.drop(columns=['agency_id', 'route_type', 'shape_id', 'date'], inplace=True)


In [13]:
first_id

,Unnamed: 0.1,Unnamed: 0,service_id,trip_id,trip_headsign,direction_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,stop_headsign,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,weekday
93788,93788,93788,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,15304,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93789,93789,93789,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,15804,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93790,93790,93790,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,20306,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93791,93791,93791,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,20805,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93792,93792,93792,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,21305,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93793,93793,93793,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,21804,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93794,93794,93794,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,22305,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93795,93795,93795,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,22804,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93796,93796,93796,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,23304,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday
93797,93797,93797,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,697,NaN,2023-07-03 03:48:00,23804,...,NaN,Bonn Dransdorf,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,Monday


In [14]:
# sort 
first_id_sort_by_time = first_id.sort_values(by='scheduled_arrival_time')


In [15]:
# look at specfici stop id
stop_id = first_id_sort_by_time[first_id_sort_by_time['stop_id'] == 7131]

In [16]:
stop_id

,Unnamed: 0.1,Unnamed: 0,service_id,trip_id,trip_headsign,direction_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,stop_headsign,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,weekday


In [17]:
sorted_stop_id = stop_id.sort_values(by=['scheduled_arrival_time', 'vrs_timestamp'])

In [18]:
sorted_stop_id

,Unnamed: 0.1,Unnamed: 0,service_id,trip_id,trip_headsign,direction_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,stop_headsign,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,weekday


In [19]:
# only look at a specific time
spec_time= sorted_stop_id[sorted_stop_id['scheduled_arrival_time'] == '2023-07-03 05:04:00']

In [20]:
spec_time

,Unnamed: 0.1,Unnamed: 0,service_id,trip_id,trip_headsign,direction_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,stop_headsign,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,weekday


In [21]:
spec_time.sort_values(by='vrs_timestamp', ascending=False)

,Unnamed: 0.1,Unnamed: 0,service_id,trip_id,trip_headsign,direction_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,stop_headsign,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,weekday


In [22]:
spectime = spec_time.drop_duplicates(subset=['trip_id', 'stop_id', 'scheduled_arrival_time'], keep='last')

In [23]:
spectime

,Unnamed: 0.1,Unnamed: 0,service_id,trip_id,trip_headsign,direction_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,stop_headsign,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,weekday


As we can see, there are duplicated columns, that only differ in vrs_timestamp. We need to keep the row with the biggest value in vrs_timestamp, since it represents the latest information about the trip. 

Now: need to develop algorithm to check for duplicates in the specified columns and checking for the biggest value.

Idea: only look at columes used above. Sort the main dataframe by trip_id, scheduled_arrival_time, stop_id, and vrs_timestamp.
Run through the rows and check if charactersics are same, except for vrs_timestamp. For it, check if the true statements in the next line a given and if the timestamp is bigger. If true, keep going, if false, we found the biggest stamp.

Either delete the "current" row if a bigger one was found or search until found and only keep it. 

## Actual 

In [25]:
data_sorted = data.sort_values(by=['trip_id', 'stop_id', 'scheduled_arrival_time', 'vrs_timestamp'])

In [26]:
data_sorted

,Unnamed: 0.1,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,...,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday,transfer_stop
93788,93788,93788,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
93789,93789,93789,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
93790,93790,93790,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
93791,93791,93791,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
93792,93792,93792,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115964,115964,115964,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.119586000000005 50.751904),"POLYGON ((7.122338292584601 50.75223811660463,...",2023-07-02 19:48:00,2023-07-02 19:48:00,0 days,0 days,1,2023-07-02,Sunday,False
115970,115970,115970,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.119586000000005 50.751904),"POLYGON ((7.122338292584601 50.75223811660463,...",2023-07-02 19:48:00,2023-07-02 19:48:00,NaN,NaN,1,2023-07-02,Sunday,False
115976,115976,115976,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.119586000000005 50.751904),"POLYGON ((7.122338292584601 50.75223811660463,...",2023-07-02 19:48:00,2023-07-02 19:48:00,NaN,NaN,1,2023-07-02,Sunday,False
115982,115982,115982,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.119586000000005 50.751904),"POLYGON ((7.122338292584601 50.75223811660463,...",2023-07-02 19:48:00,2023-07-02 19:48:00,NaN,NaN,1,2023-07-02,Sunday,False


In [27]:

#data_clean = data_sorted.sort_values('vrs_timestamp', ascending=False)
data_clean = data_sorted.drop_duplicates(subset=['trip_id', 'stop_id', 'scheduled_arrival_time'], keep='last')

In [28]:
data_clean

,Unnamed: 0.1,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,...,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday,transfer_stop
93805,93805,93805,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,...,POINT (7.047870000000005 50.73739700000001),"POLYGON ((7.050620588160284 50.73773274040921,...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
65490372,65490372,65490372,537,8,537,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,64,1-537-008-2140.2.21:043200-36-1_5FB81E38-1A0D-...,...,POINT (7.097127999999999 50.732008),POLYGON ((7.0998788594577125 50.73234255286602...,2023-07-03 05:23:00,2023-07-03 05:23:00,0 days,NaN,1,2023-07-03,Monday,False
65491200,65491200,65491200,537,8,537,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,64,1-537-008-2140.2.21:043200-36-1_5FB81E38-1A0D-...,...,POINT (7.097127999999999 50.732008),POLYGON ((7.0998788594577125 50.73234255286602...,2023-07-04 05:23:00,2023-07-04 05:23:00,0 days,NaN,1,2023-07-04,Tuesday,False
65491982,65491982,65491982,537,8,537,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,64,1-537-008-2140.2.21:043200-36-1_5FB81E38-1A0D-...,...,POINT (7.097127999999999 50.732008),POLYGON ((7.0998788594577125 50.73234255286602...,2023-07-05 05:23:00,2023-07-05 05:23:00,0 days,NaN,1,2023-07-05,Wednesday,False
65492787,65492787,65492787,537,8,537,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,64,1-537-008-2140.2.21:043200-36-1_5FB81E38-1A0D-...,...,POINT (7.097127999999999 50.732008),POLYGON ((7.0998788594577125 50.73234255286602...,2023-07-06 05:23:00,2023-07-06 05:23:00,0 days,NaN,1,2023-07-06,Thursday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115985,115985,115985,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.097651999999999 50.736282),"POLYGON ((7.1004031160749 50.736616560650745, ...",2023-07-02 19:39:00,2023-07-02 19:39:00,NaN,NaN,1,2023-07-02,Sunday,False
115984,115984,115984,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.094874000000002 50.733772999999985),POLYGON ((7.097624936034717 50.734107614367865...,2023-07-02 19:37:00,2023-07-02 19:37:00,NaN,NaN,1,2023-07-02,Sunday,True
115986,115986,115986,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.102483000000003 50.73746799999999),POLYGON ((7.1052342430250555 50.73780245225033...,2023-07-02 19:41:00,2023-07-02 19:41:00,NaN,NaN,1,2023-07-02,Sunday,True
115987,115987,115987,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,...,POINT (7.117441999999999 50.739231),"POLYGON ((7.120193524138334 50.73956510756546,...",2023-07-02 19:44:00,2023-07-02 19:44:00,NaN,NaN,1,2023-07-02,Sunday,True


In [29]:
from shapely.wkt import loads

In [30]:
data_clean['station_point'] = data_clean['station_point'].apply(shapely.wkt.loads)

/tmp/ipykernel_13640/516118530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['station_point'] = data_clean['station_point'].apply(shapely.wkt.loads)


In [31]:
gdf = gpd.GeoDataFrame(data_clean, geometry='station_point')

In [32]:
#gdf.to_parquet('/Volumes/T7/Master/Processed Data/'+data_month_double+'/11_data_clean.parquet', engine='pyarrow')

In [33]:
#gdf.to_csv('../Data/11_data_clean.csv')
gdf.to_csv('../Data/Data/10_VRS_clean.csv')